In [1]:
import pandas as pd

TESLA = '/kaggle/input/tesla-stock-data-updated-till-28jun2021/TSLA.csv'

df = pd.read_csv(filepath_or_buffer=TESLA, parse_dates=['Date'])
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,3.800,5.000,3.508,4.778,4.778,93831500
1,2010-06-30,5.158,6.084,4.660,4.766,4.766,85935500
2,2010-07-01,5.000,5.184,4.054,4.392,4.392,41094000
3,2010-07-02,4.600,4.620,3.742,3.840,3.840,25699000
4,2010-07-06,4.000,4.000,3.166,3.222,3.222,34334500


In [2]:
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
2951,2022-03-18,874.489990,907.849976,867.390015,905.390015,905.390015,33408500
2952,2022-03-21,914.979980,942.849976,907.090027,921.159973,921.159973,27327200
2953,2022-03-22,930.000000,997.859985,921.750000,993.979980,993.979980,35289500
2954,2022-03-23,979.940002,1040.699951,976.400024,999.109985,999.109985,40225400
2955,2022-03-24,1009.729980,1024.489990,988.799988,1013.919983,1013.919983,22901900


In [3]:
from plotly.express import scatter
scatter(data_frame=df, x='Date', y=['Open', 'High', 'Low', 'Close', 'Adj Close'], log_y=True, )

In [4]:
# total return
total_return = 100 * df['Adj Close'].values[-1] / df['Adj Close'].values[0]
buy_and_hold_profit = df['Adj Close'].values[-1] - df['Adj Close'].values[0]
print('total return: {} pct'.format(round(100 * total_return)/100))
print('buy and hold profit: {:5.2f}'.format(buy_and_hold_profit))

total return: 21220.59 pct
buy and hold profit: 1009.14


Let's find the annualized return.

In [5]:
# years
years = (df['Date'].max() - df['Date'].min()).days / 365.25
print('years: {}'.format(round(100 * years)/100))

years: 11.73


In [6]:
from math import log
from math import exp

def get_rate(start_value: float, end_value: float, years: float) -> float:
    rate = exp(1.0/years * log(end_value/start_value)) - 1.0
    return rate

annualized_rate = get_rate(start_value=df['Adj Close'].values[0], end_value=df['Adj Close'].values[-1], years=years)

print('annualized pct rate: {}'.format(100 * round(10000 * annualized_rate) / 10000))

annualized pct rate: 57.86


This implies we would make 58% annually if we bought at the adjusted close the first day of trading and held until the last day in our dataset.

In [7]:
total = df['Adj Close'].values[0]
totals = [total]
rate = 0.5786
for year in range(1, 12):
    total = (1 + rate) * total 
    totals.append(total)
    print('year: {} : {:5.2f}'.format(year, total))
total = (1 + rate * 0.73) * total
print('year: {} : {:5.2f}'.format(year + .73, total))

year: 1 :  7.54
year: 2 : 11.91
year: 3 : 18.80
year: 4 : 29.67
year: 5 : 46.84
year: 6 : 73.94
year: 7 : 116.72
year: 8 : 184.26
year: 9 : 290.87
year: 10 : 459.16
year: 11 : 724.84
year: 11.73 : 1030.99


We're off by eighteen dollars over nearly 12 years, so it's safe to say our estimate of a 58% annualized return is pretty close.

Let's look at what we would make instead if we used a simple moving average strategy where we track the 50-day and 200-day moving averages, and we buy when the 50-day average rises above the 200-day moving average and sell when it goes below.

In [8]:
from plotly.express import line

close_df = df[['Date', 'Adj Close']].copy()
close_df['50d'] = close_df['Adj Close'].rolling(window=50).mean()
close_df['200d'] = close_df['Adj Close'].rolling(window=200).mean()
line(data_frame=close_df, x='Date', y=['50d', '200d'], log_y=True).show()

From the looks of things we will not make a lot of trades. We will make a trade every time the two lines above cross, or when the difference between our two moving averages switches signs.

In [9]:
import math
close_df['sign'] = close_df.apply(axis=1, func=lambda x: math.copysign(1, x['50d'] - x['200d']))
line(data_frame=close_df, x='Date', y='sign', height=400).show()

Let's compute our profit subject to the assumptions:
* We borrow our initial stake and pay no interest on the amount we borrow
* We do not try to estimate our moving averages before they are available
* We estimate our closing profit off the final price as if we exited the trade on the last day for which we have data
* We ignore transaction costs and taxes

In [10]:
profits = []
state = 0
profit = 0
count = 0
plot_df = close_df.dropna().copy() # we can't trade until our moving averages are valid
for index, row in plot_df.iterrows():
    price = row['Adj Close']
    if row['50d']  > row['200d']:
        if state == 0:
            state = 1
            profit -= price
            print('buy: {} price: {:5.2f} profit: {:5.2f}'.format(row['Date'].date(), price, profit))
            count += 1
    elif row['50d'] < row['200d']:
        if state == 1:
            state = 0
            profit += price
            print('sell: {} price:  {:5.2f} profit: {:5.2f}'.format(row['Date'].date(), price, profit))
            count += 1
    profits.append(profit)
if state == 1:
    state = 0
    count += 1
    profit += price
    print('closeout:  price: {:5.2f} profit: {:5.2f}'.format(price, profit))
print('we made {} trades'.format(count))
# do we make more money by being in in good times and out in bad times?
plot_df['profit'] = plot_df['Adj Close'] + profits
scatter(data_frame=plot_df, x='Date', y=['Adj Close', 'profit',], log_y=False).show()
scatter(data_frame=plot_df, x='Date', y=['Adj Close', 'profit',], log_y=True).show()

buy: 2011-04-12 price:  4.93 profit: -4.93
sell: 2011-08-26 price:   4.75 profit: -0.18
buy: 2011-11-03 price:  6.49 profit: -6.68
sell: 2012-07-06 price:   6.20 profit: -0.48
buy: 2012-12-20 price:  6.89 profit: -7.36
sell: 2015-01-12 price:  40.44 profit: 33.08
buy: 2015-06-09 price: 51.20 profit: -18.12
sell: 2015-11-16 price:  42.86 profit: 24.74
buy: 2016-05-02 price: 48.36 profit: -23.62
sell: 2016-07-07 price:  43.19 profit: 19.57
buy: 2016-07-21 price: 44.10 profit: -24.53
sell: 2016-10-05 price:  41.69 profit: 17.16
buy: 2017-01-31 price: 50.39 profit: -33.23
sell: 2017-12-12 price:  68.21 profit: 34.98
buy: 2018-08-02 price: 69.91 profit: -34.93
sell: 2018-09-11 price:  55.89 profit: 20.96
buy: 2018-12-03 price: 71.70 profit: -50.74
sell: 2019-02-28 price:  63.98 profit: 13.24
buy: 2019-11-05 price: 63.44 profit: -50.21
sell: 2021-07-09 price:  656.95 profit: 606.74
buy: 2021-08-30 price: 730.91 profit: -124.17
closeout:  price: 1013.92 profit: 889.75
we made 22 trades


We made money with this moving-average-based trading scheme, but we would have made more money (and encountered lower fees and taxes) if we just bought and held, but there are some periods of time during the period of interest when we do better by active trading than by buying and holding.